# Demo: encoding apollo scape via vehicle_reconstructor

## Step 1: Loading dataset

In [ ]:
import glob
import torch
from pytorch3d.io import load_obj
from pytorch3d.structures import Meshes, Pointclouds
from pytorch3d.vis.plotly_vis import plot_scene
from pprint import pprint
import os

import sys
sys.path.append("..")

from vehicle_reconstruction import vehicle, vehicle_reconstructor, constants
from vehicle_reconstruction.data_utils import apolloscape_dataset

DEVICE = torch.device("cuda:0")

constants.apolloscape_constant.set_apolloscape_dir("/mnt/sda/uzuki_space/voxeltorch/assets/apollo_scape")

In [ ]:
for id, vehi in enumerate(constants.apolloscape_constant.models):
    print(f"\t{id}:\t{vehi.name:25}", end='\t')
    if id % 4 == 0:
        print()

In [ ]:
apolloscape = apolloscape_dataset()
apolloscape_meshes = apolloscape.get_batch_centered_meshes().to(DEVICE)

In [ ]:
def visualize_meshes(meshes: Meshes, title: str = "Mesh"):
    mesh_dict = {}
    for idx in range(meshes.__len__()):
        mesh_dict[title + f" {idx}"] = {
            "mesh": meshes[idx]
        }
    fig = plot_scene(mesh_dict, ncols=2)
    fig.update_layout(height=400, width=400 * meshes.__len__())
    return fig

selected_idx = torch.randint(0, apolloscape_meshes.__len__() - 1, (2, ))
print("Visualizing original meshes...")
fig = visualize_meshes(apolloscape_meshes[selected_idx], "Original Meshes")
fig.show()

## Step 2: Preparing vehicle reconstructor

In [ ]:
TSDF_UNIT = torch.Tensor([0.1]*3).to(DEVICE)
bbox = apolloscape.get_bbox().to(DEVICE)

antisotropic_res = (bbox / TSDF_UNIT).ceil().int()

vehi_reconstructor = vehicle_reconstructor(resolution=antisotropic_res, 
                      bbox=bbox)

In [ ]:
vehi_reconstructor.fit_meshes(apolloscape_meshes)
vehi_reconstructor.save_parameters("../data/vehicle_ckpt.pt")

## Step 3: Testing latent reconstruction

In [ ]:
tsdf_grid = vehi_reconstructor.prepare_tsdf(apolloscape_meshes)
reconstructed_tsdf_grids = vehi_reconstructor.reconsturct(tsdf_grid)

In [ ]:
from voxeltorch import TSDF, tsdf2meshes

selected_idx = torch.randint(0, apolloscape_meshes.__len__() - 1, (2, ))

print(f"Randomly selected index: [{selected_idx}]")
print("Visualizing voxel meshes from TSDF...")
tsdf_meshes = tsdf2meshes(
    tsdf_grid[selected_idx], TSDF_UNIT)
fig = visualize_meshes(tsdf_meshes, "Original TSDF Meshes")
fig.show()

print("Visualizing voxel meshes from reconstructed TSDF...")
tsdf_meshes = tsdf2meshes(
    reconstructed_tsdf_grids[selected_idx], TSDF_UNIT)
fig = visualize_meshes(tsdf_meshes, "Reconstructed TSDF Meshes")
fig.show()

## Step 4: Exploring latent space

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, FloatSlider

global vehi
vehi = None

def slidar_create(min=-10.0, max=10.0, desc="desc:"):
    return widgets.FloatSlider(
        value=0.0,
        min=min,
        max=max,
        step=0.01,
        description=desc,
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='.2f',
    )


def generate_model(a, b, c, d, e):
    latent = torch.Tensor([[a, b, c, d, e]]).cuda()
    global vehi
    vehi = vehi_reconstructor.decode(latent)
    print("Visualizing voxel meshes from reconstructed TSDF...")
    tsdf_meshes = tsdf2meshes(
        vehi, TSDF_UNIT)
    fig = visualize_meshes(tsdf_meshes, "Reconstructed TSDF Meshes")
    fig.show()
    # pm.create_figure_for_inspection(
    #     [vehi.vertices[:, [0, 2, 1]]], [vehi.faces], f"temp.html")

interact(generate_model, a=slidar_create(-1.0, 1.0, 'v'),
         b=slidar_create(-1.0, 1.0, 'w'),
         c=slidar_create(-1.0, 1.0, 'x'),
         d=slidar_create(-1.0, 1.0, 'y'),
         e=slidar_create(-1.0, 1.0, 'z'))